In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0027'
spoofed = 'spoofed'
MAX_BIAS = 50

if kitti_seq == '0028':
    start_idx = 1550
else:
    start_idx = 0

In [ ]:
# Load results
results_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'results', spoofed)
results = np.load(os.path.join(results_path, '2023-01-17_2252_fgo_50m.npz'))

# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

graph_positions_plot = results['positions']
spoofed_positions_plot = results['spoofed']
qs_plot = results['qs']
gt_enu_plot = gt_enu[:graph_positions_plot.shape[0]]

Trajectory Plots

In [ ]:
fgo_traj = go.Scatter(x=graph_positions_plot[:,0], y=graph_positions_plot[:,1], name='FGO')
gt_traj = go.Scatter(x=gt_enu_plot[:,0], y=gt_enu_plot[:,1], name='Ground-truth', line=dict(color='black'))
spoof_traj = go.Scatter(x=spoofed_positions_plot[:,0], y=spoofed_positions_plot[:,1], name='Spoofed', line=dict(color='red', dash='dash')) 
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[fgo_traj, gt_traj, spoof_traj, start])
fig.update_layout(width=700, height=700, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

In [ ]:
T = 150

# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='red', dash='dash')))
# Add vertical line at start of spoofing attack
#fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
#fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.show()

Error Plots

Error values